# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [2]:
data=pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Итак. В нашем распоряжение есть данные о 10 тысячах клиентах банка. Нам нужно создать наиболее эффективную модель, которая будет выявлять уйдёт ли клиент от этого банка. Для начала данные нужно подготовить для создания моделей. 

### Выбираем нужные признаки

Так как нам нужно создать модель дающую два взаимоисключающих ответа, то у нас следовательно задача классификации. Для такой задачи из представленных данных нам нужно будет выбрать целевой признак и признаки на основе которых будет происходить прогнозирование нашей модели. 

Для начала опишем все имеющиеся у нас признаки, т.е. столбцы нашего датафрейма:

    RowNumber — индекс строки в данных
    CustomerId — уникальный идентификатор клиента
    Surname — фамилия
    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата
    Exited — факт ухода клиента

Мы сразу можем отметить, что последний признак Exited является целевым. Проверим какие значения хранит этот столбец. 

In [5]:
data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

С целевым признаком всё впорядке, лишних значений нет. Из остальных признаков мы считаем нужным сразу удалить из нашего датафрейма:

	RowNumber — индекс строки в данных, так как у нас есть обычные индексы, а эти числовые значения будут путать модель
	CustomerId — уникальный идентификатор клиента, так как эти числовые значения будут путать модель. 

In [6]:
data=data.drop(['RowNumber'], axis=1)
data=data.drop(['CustomerId'], axis=1)

### Признак Surname  - фамилия.

Сложно сказать однозначно, влияет ли фамилия на уход клиентов. С одной стороны, часть пользователей банковских услуг живут семьями, и многим скорее всего удобно использовать один банк для общего счёта, общей ипотеки, общей слежки за тем как пользуется своей банковской картой несовершеннолетний ребёнок и так далее. Вполне возможно что у конкурентов нашего банка есть семейные более удобные программы. Поэтому связь между фамилиями может быть. С другой стороны, у семей не индивидуальные фамилии, у нас есть однофамильцы не являющиеся нам супругами и вообще родственниками. Поэтому это всё будет путать нашу модель. На всякий случай посмотрим сколько фамилий в наших данных.

In [7]:
data['Surname'].nunique()

2932

Фамилий много. Если мы попытаемся обработать эти данные для работы модели, данные увеличатся во много раз. А фактор того что у семей не индивидуальные фамилии лишь усугубит ситуацию. Поэтому мы для нашей работы исключаем столбец Surname с фамилиями из данных. 

Сделаем оговорку. Проследить связь между семьями в нашем вопросе скорее всего возможно, но для этого нам нужны данные завязанные на времени и и более детальных данных о регионе проживания. То есть имея данные, например за 3-4 месяцев, проследить как часто повторялась одна фамилия от месяца к месяцу. Однако это уже другое исследование, с другими данными и подзадачами.

In [8]:
data=data.drop(['Surname'], axis=1)

### Проблема пропусков

В наших данных в столбце Tenure (сколько лет человек является клиентом банка) существуют пропуски. Мы можем либо удалить их либо заполнить средним или медианным значением. Если мы удалим их, то наша модель будет мало применима в будущем уже на других данных, где также могут быть пропуски. Поэтому надо заполнить. Заполнять будем медианным значением, так как медиана более адекватный показатель чем среднее.

In [9]:
median_Tenure = data['Tenure'].median()
data['Tenure']=data['Tenure'].fillna(value=median_Tenure)

### Обработка признаков

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Итак для нашей модели мы выбрали следующие не целевые признаки: 

    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата
	
Типы данных соответствуют тому, что по логике должно храниться в данных. Geography и Gender это категориальные данные. Которые нам нужно будет преобразовать. Прежде посмотрим сколько уникальных значений хранится в каждом признаке. 

In [11]:
data['Geography'].nunique()

3

In [12]:
data['Gender'].nunique()

2

Уникальных значений не так много. Поэтому преобразуем данные техникой техникой OHE, используя также параметр drop_first=True, дабы избежать дамми ловушки, скинув первый образовавшийся столбец. 

**Дабы избежать ошибок, преобразовывать данные мы будем после того как разобьём их на выборки**

Также нам нужно будет масштабировать количественные признаки у которых наблюдаются большие показатели. Мы считаем что масштабировать следует следующие признаки: CreditScore,  Age, Tenure, Balance, EstimatedSalary. Другие признаки не обладают большими показателями исходя из таблицы, но проверим их.

In [13]:
data['IsActiveMember'].unique()

array([1, 0])

In [14]:
data['NumOfProducts'].unique()

array([1, 3, 2, 4])

In [15]:
data['HasCrCard'].unique()

array([1, 0])

Признаки IsActiveMember, NumOfProducts, HasCrCard оставим как есть. 
Масштабируем признаки: CreditScore, Age, Tenure, Balance, EstimatedSalary.

Но прежде чем провести масштабирование создадим выборки. 

### Создадим выборки

В своей работе мы создадим три модели, будем использовать для них следующие названия: решающее дерево, случайный лес и логистическая регрессия.

Все имеющиеся у нас данные мы разделим на три части: обучающую, валидационную и тестовую выборку. С помощью первых двух мы построим модели с наиболее эффективными гиперпараметрами. А с помощью тестовой выборки совершим финальную проверку наилучшей модели созданной с помощью обучающей и валидационной выборок.   

Разбивая данные на обучающую, валидационную и тестовую выборку, мы будем следовать совету из теории, и разобьём данные в соотношение 3:1:1.

Сначала разобьём наш датафрейм на целевой (target) и все остальные признаки (features).

In [16]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']

Теперь разобьём данные на для создания моделей (work) и для конечной проверки моделей (test), в последнюю категорию отнесём 20%, как советует теория.

In [17]:
features_work, features_test, target_work, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

Теперь разобьём данные для создания моделей (work) на для первоначального создания моделей (train) и на валидационные для их подкручивания гиперпараметрами (valid), в последнюю категорию отнесём 25%, как советует теория.

In [18]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_work, target_work, test_size=0.25, random_state=12345)

Итак, мы разбили все данные на три выборки:

    features_train, target_train - для первоначального создания моделей
    features_valid, target_valid - валидационные для подкручивания гиперпараметров
    features_test, target_test - тестовые, для финальной проверки

### Масштабирование

In [19]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

StandardScaler()

In [20]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None

In [21]:
features_train.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
492,-0.134048,France,Female,-0.078068,-0.369113,0.076163,2,0,1,0.331571
6655,-1.010798,France,Male,0.494555,-0.007415,0.136391,1,1,1,-0.727858
4287,0.639554,Germany,Male,1.353490,-1.454209,0.358435,1,1,1,-0.477006


In [22]:
features_valid.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
2358,0.175393,France,Male,0.399118,-1.454209,1.385698,1,0,1,-1.466761
8463,-1.299609,Spain,Male,0.971741,-1.092511,-1.232442,1,1,0,0.254415
163,0.711757,Spain,Female,-0.268942,-1.092511,-1.232442,2,1,1,0.122863


In [23]:
features_test.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7867,-0.123733,Spain,Female,0.685430,-0.730812,-1.232442,1,1,1,0.980212
1402,1.083087,France,Male,-0.937002,1.077681,0.858518,1,1,0,-0.390486
8606,1.598822,Spain,Male,0.303681,-0.007415,-1.232442,2,1,1,-0.435169


Всё прошло успешно. Теперь приступим к следующему этапу. 

### Преобразуем техникой OHE

Преобразуем данные техникой OHE, используя также параметр drop_first=True, дабы избежать дамми ловушки, скинув первый образовавшийся столбец.

In [24]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_train.shape
features_train.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
492,-0.134048,-0.078068,-0.369113,0.076163,2,0,1,0.331571,0,0,0
6655,-1.010798,0.494555,-0.007415,0.136391,1,1,1,-0.727858,0,0,1
4287,0.639554,1.353490,-1.454209,0.358435,1,1,1,-0.477006,1,0,1


In [25]:
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_valid.shape
features_valid.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2358,0.175393,0.399118,-1.454209,1.385698,1,0,1,-1.466761,0,0,1
8463,-1.299609,0.971741,-1.092511,-1.232442,1,1,0,0.254415,0,1,1
163,0.711757,-0.268942,-1.092511,-1.232442,2,1,1,0.122863,0,1,0


In [26]:
features_test = pd.get_dummies(features_test, drop_first=True)
features_test.shape
features_test.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7867,-0.123733,0.685430,-0.730812,-1.232442,1,1,1,0.980212,0,1,0
1402,1.083087,-0.937002,1.077681,0.858518,1,1,0,-0.390486,0,0,1
8606,1.598822,0.303681,-0.007415,-1.232442,2,1,1,-0.435169,0,1,1


OHE сработало. Приступаем к следующему этапу.

## Исследование задачи

Создавая модель мы будем стремимся к наибольшему значению F1-меры, минимальный пороговый показатель этой метрики установим на 0.59. Дополнительно будем измерять AUC-ROC, и сравнивать её значение с F1-мерой.

### Решающее дерево

Напишем цикл, который создаст модель решающее дерево и проверим эффективность (через значение F1-меры) у разного значения гиперпараметра глубины (max_depth) от 1 до 15.

In [27]:
for i in range(1,16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.0 : 0.6904695296120447
max_depth = 2 : 0.5037037037037037 : 0.7396462354498909
max_depth = 3 : 0.39382239382239387 : 0.7938649126794771
max_depth = 4 : 0.430188679245283 : 0.8064340768598627
max_depth = 5 : 0.5488372093023256 : 0.8224509194603883
max_depth = 6 : 0.5113043478260869 : 0.8136862819275844
max_depth = 7 : 0.5583596214511041 : 0.8231010349393358
max_depth = 8 : 0.5398773006134968 : 0.7996889300752322
max_depth = 9 : 0.5357142857142857 : 0.7814086047313784
max_depth = 10 : 0.5383502170767005 : 0.7580608756054101
max_depth = 11 : 0.5131964809384164 : 0.7388991589826408
max_depth = 12 : 0.508029197080292 : 0.72204066321316
max_depth = 13 : 0.49657064471879286 : 0.6930707862900343
max_depth = 14 : 0.5079787234042553 : 0.6940872871428139
max_depth = 15 : 0.4993288590604027 : 0.6926694313794369


На данном этапе не одно из древ не достигло порогового для нас значения f1 ( 0.59).

### Случайный лес

Напишем циклы, которые создут модель случайный лес и проверим эффективность (через значение F1-меры) у разного значения гиперпараметра количества деревьев (n_estimators и max_depth).

In [28]:
for i in range(1,25):
    model = RandomForestClassifier(random_state=12345,max_depth=7, n_estimators=i)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'n_estimators = {i} : {f1} : {auc_roc}')

n_estimators = 1 : 0.40211640211640215 : 0.7723220885078975
n_estimators = 2 : 0.4659498207885305 : 0.7860349154929354
n_estimators = 3 : 0.5016949152542374 : 0.8003485827005701
n_estimators = 4 : 0.4931972789115647 : 0.8063553954021417
n_estimators = 5 : 0.5051194539249146 : 0.814320502162548
n_estimators = 6 : 0.5034722222222222 : 0.8202144586318327
n_estimators = 7 : 0.515358361774744 : 0.8273657289002558
n_estimators = 8 : 0.5163511187607573 : 0.8273156588817061
n_estimators = 9 : 0.5085910652920963 : 0.8309159316440926
n_estimators = 10 : 0.5227655986509274 : 0.8305106029224996
n_estimators = 11 : 0.5261382799325464 : 0.8330959643565048
n_estimators = 12 : 0.5286195286195287 : 0.8330363571915648
n_estimators = 13 : 0.525963149078727 : 0.8328464090259553
n_estimators = 14 : 0.53 : 0.8334544021083452
n_estimators = 15 : 0.5487603305785124 : 0.8354961461980961
n_estimators = 16 : 0.5409015025041736 : 0.8366461671003419
n_estimators = 17 : 0.5433333333333333 : 0.8382484076939338
n_est

Лучший показатель f1 (0.548) у n_estimators=15

In [29]:
for i in range(1,21):
    model = RandomForestClassifier(random_state=12345,max_depth=i, n_estimators=15)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.0 : 0.7818814882399038
max_depth = 2 : 0.22022471910112357 : 0.7873573997924082
max_depth = 3 : 0.23660714285714285 : 0.8066152826412809
max_depth = 4 : 0.3483606557377049 : 0.8131919398396805
max_depth = 5 : 0.4782608695652174 : 0.8261028517657233
max_depth = 6 : 0.5034965034965035 : 0.8343477148202486
max_depth = 7 : 0.5487603305785124 : 0.8354961461980961
max_depth = 8 : 0.534453781512605 : 0.8421093624576589
max_depth = 9 : 0.5433715220949265 : 0.836112881664677
max_depth = 10 : 0.5436893203883495 : 0.8415999198879702
max_depth = 11 : 0.5616883116883117 : 0.8359849249506056
max_depth = 12 : 0.5537974683544303 : 0.8336260707433728
max_depth = 13 : 0.54858934169279 : 0.8297102773879028
max_depth = 14 : 0.5486443381180224 : 0.8180988016575561
max_depth = 15 : 0.5479876160990712 : 0.826679054360145
max_depth = 16 : 0.5485362095531587 : 0.8178468620404089
max_depth = 17 : 0.5425867507886436 : 0.8177045996067518
max_depth = 18 : 0.557427258805513 : 0.8205601801884858
ma

На данном этапе лучший лес (f1=0.56) имеет гиперпараметры max_depth=11 и n_estimators=15. У этих гиперпараметров auc_roc=0.835. Лучший же auc_roc (0.84) имеет max_depth=8. 

### Логистическая регрессия

Напишем цикл, который создаст модель логистической регрессии и проверим эффективность (через значение  f1) у разного значения гиперпараметра максимального количества итераций обучения (max_iter) от 100 до 1500.

In [30]:
for i in range(100,1700,100):
    model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=i)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_iter = {i} : {f1} : {auc_roc}')

max_iter = 100 : 0.30131826741996237 : 0.7703773054064493
max_iter = 200 : 0.30131826741996237 : 0.7703773054064493
max_iter = 300 : 0.30131826741996237 : 0.7703773054064493
max_iter = 400 : 0.30131826741996237 : 0.7703773054064493
max_iter = 500 : 0.30131826741996237 : 0.7703773054064493
max_iter = 600 : 0.30131826741996237 : 0.7703773054064493
max_iter = 700 : 0.30131826741996237 : 0.7703773054064493
max_iter = 800 : 0.30131826741996237 : 0.7703773054064493
max_iter = 900 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1000 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1100 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1200 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1300 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1400 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1500 : 0.30131826741996237 : 0.7703773054064493
max_iter = 1600 : 0.30131826741996237 : 0.7703773054064493


Очевидно что пока на данном этапе логистическая регрессия слабо справляется с поставленной задачей.  

## Борьба с дисбалансом

На прошлых этапах ни одна из моделей не достигла порогового для нас значения f1 ( 0.59). Возможно это связано с дисбалансом классов. 

In [31]:
data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Классы соотнесены друг к другу примерно 4:1. Попробуем разные техники для устранения дисбаланса. 

### Придание большего веса

Попробуем придадим объектам редкого класса больший вес через введение параметра class_weight='balanced' в наши модели. 

Проверим решающее дерево.

In [32]:
for i in range(1,16):
    model = DecisionTreeClassifier(random_state=12345, 
                                   max_depth=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.4750733137829912 : 0.6898591522430575
max_depth = 2 : 0.49761677788369874 : 0.7381902311009524
max_depth = 3 : 0.49761677788369874 : 0.79092270301803
max_depth = 4 : 0.5170454545454546 : 0.8064610987746356
max_depth = 5 : 0.5489078822412156 : 0.818801371441651
max_depth = 6 : 0.5587044534412956 : 0.8090671240258203
max_depth = 7 : 0.5535353535353534 : 0.8058801276070185
max_depth = 8 : 0.541622760800843 : 0.7895803496635772
max_depth = 9 : 0.5145631067961165 : 0.7543366199399478
max_depth = 10 : 0.4925816023738872 : 0.7259858627700007
max_depth = 11 : 0.47782258064516125 : 0.7169605432358584
max_depth = 12 : 0.48370136698212407 : 0.7144379680155901
max_depth = 13 : 0.47982551799345696 : 0.7099253082485191
max_depth = 14 : 0.484304932735426 : 0.700245104662234
max_depth = 15 : 0.4836759371221282 : 0.6977137870577744


Пороговое значения f1 ( 0.59) не достигнуто. 

Проверим случайный лес.

In [33]:
for i in range(1,21):
    model = RandomForestClassifier(random_state=12345,max_depth=7, 
                                   n_estimators=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'n_estimators = {i} : {f1} : {auc_roc}')

n_estimators = 1 : 0.5208333333333333 : 0.7844771815824988
n_estimators = 2 : 0.512118018967334 : 0.7889485137152115
n_estimators = 3 : 0.5441020191285867 : 0.8071366466439578
n_estimators = 4 : 0.5550755939524837 : 0.8158361136764268
n_estimators = 5 : 0.5590200445434298 : 0.8157192836331442
n_estimators = 6 : 0.564885496183206 : 0.8216458253525962
n_estimators = 7 : 0.553980370774264 : 0.8264517523711731
n_estimators = 8 : 0.5620915032679739 : 0.8338120450979862
n_estimators = 9 : 0.5609492988133764 : 0.8383636482128183
n_estimators = 10 : 0.5655021834061135 : 0.8351353241596582
n_estimators = 11 : 0.5685840707964601 : 0.8352036737087896
n_estimators = 12 : 0.5701657458563536 : 0.8343763262594199
n_estimators = 13 : 0.5717439293598233 : 0.8363997908185892
n_estimators = 14 : 0.5635964912280701 : 0.8374449031105402
n_estimators = 15 : 0.5593035908596301 : 0.8388564007763237
n_estimators = 16 : 0.5676567656765676 : 0.8402249812833502
n_estimators = 17 : 0.5726970033296338 : 0.841852654

Лучший n_estimators = 18 (F1=0.57)

In [34]:
for i in range(1,15):
    model = RandomForestClassifier(random_state=12345,max_depth=i, 
                                   n_estimators=18, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.5175983436853002 : 0.7697065261103226
max_depth = 2 : 0.5142857142857142 : 0.8056027555994971
max_depth = 3 : 0.5330739299610894 : 0.819457050255993
max_depth = 4 : 0.5471698113207547 : 0.8252619933589671
max_depth = 5 : 0.5686486486486485 : 0.8387872564649931
max_depth = 6 : 0.5609756097560975 : 0.8370650067793215
max_depth = 7 : 0.5736607142857142 : 0.8431719595179926
max_depth = 8 : 0.5879629629629629 : 0.8485334253138117
max_depth = 9 : 0.5717761557177615 : 0.8461197325148344
max_depth = 10 : 0.5813060179257363 : 0.8427070236314591
max_depth = 11 : 0.576271186440678 : 0.8388619641117181
max_depth = 12 : 0.5511596180081855 : 0.831643139056363
max_depth = 13 : 0.5517241379310345 : 0.8187044104533483
max_depth = 14 : 0.5663189269746648 : 0.8146630446704042


Лучший max_depth = 8 (F1=0.588). Но пороговое значения f1 ( 0.59) не достигнуто.

Проверим логистическую регрессию.  

In [35]:
for i in range(100,1700,100):
    model = LogisticRegression(random_state=12345, solver='liblinear', 
                               max_iter=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_iter = {i} : {f1} : {auc_roc}')

max_iter = 100 : 0.4741532976827095 : 0.7725581328810607
max_iter = 200 : 0.4741532976827095 : 0.7725581328810607
max_iter = 300 : 0.4741532976827095 : 0.7725581328810607
max_iter = 400 : 0.4741532976827095 : 0.7725581328810607
max_iter = 500 : 0.4741532976827095 : 0.7725581328810607
max_iter = 600 : 0.4741532976827095 : 0.7725581328810607
max_iter = 700 : 0.4741532976827095 : 0.7725581328810607
max_iter = 800 : 0.4741532976827095 : 0.7725581328810607
max_iter = 900 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1000 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1100 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1200 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1300 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1400 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1500 : 0.4741532976827095 : 0.7725581328810607
max_iter = 1600 : 0.4741532976827095 : 0.7725581328810607


Пороговое значения f1 ( 0.59) не достигнуто. 

### Техника upsampled

Увеличим количество положительных объектов в 3.9 раза чтобы выровнять баланс объектов. 

In [36]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

features_upsampled и target_upsampled - новые наборы обучающих данных. 

Проверим решающее дерево. 

In [37]:
for i in range(1,21):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.4750733137829912 : 0.6898591522430575
max_depth = 2 : 0.49761677788369874 : 0.7381902311009524
max_depth = 3 : 0.49761677788369874 : 0.79092270301803
max_depth = 4 : 0.5170454545454546 : 0.8064610987746356
max_depth = 5 : 0.5489078822412156 : 0.818801371441651
max_depth = 6 : 0.5587044534412956 : 0.809054407830633
max_depth = 7 : 0.5549949545913219 : 0.8083621699551278
max_depth = 8 : 0.545068928950159 : 0.7867486119478191
max_depth = 9 : 0.5136186770428015 : 0.7499757597529243
max_depth = 10 : 0.48538011695906436 : 0.7229689454618284
max_depth = 11 : 0.48790746582544686 : 0.7196047170726048
max_depth = 12 : 0.48144220572640506 : 0.7078668741525848
max_depth = 13 : 0.47859495060373214 : 0.7092759875317706
max_depth = 14 : 0.4865470852017937 : 0.702836824193833
max_depth = 15 : 0.46245530393325385 : 0.685878983149452
max_depth = 16 : 0.47699757869249393 : 0.6896159550101015
max_depth = 17 : 0.4671532846715329 : 0.674895369556475
max_depth = 18 : 0.48431618569636137 : 0

Пороговое значения f1 ( 0.59) не достигнуто. Лучший показатель у max_depth = 6 (f1=0.55).

Проверим случайный лес.

In [38]:
for i in range(1,16):
    model = RandomForestClassifier(random_state=12345,max_depth=6, 
                                   n_estimators=i)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'n_estimators = {i} : {f1} : {auc_roc}')

n_estimators = 1 : 0.5511811023622047 : 0.7926211098377255
n_estimators = 2 : 0.513170731707317 : 0.7968134804385181
n_estimators = 3 : 0.53220696937698 : 0.8073083152789853
n_estimators = 4 : 0.5251282051282051 : 0.8147846432868822
n_estimators = 5 : 0.5561613958560523 : 0.8214304447966123
n_estimators = 6 : 0.5590200445434298 : 0.8270287497277939
n_estimators = 7 : 0.5720524017467249 : 0.8306989615637105
n_estimators = 8 : 0.5595744680851064 : 0.8334933454561061
n_estimators = 9 : 0.5621621621621622 : 0.840507121864067
n_estimators = 10 : 0.5585392051557464 : 0.8387522869282282
n_estimators = 11 : 0.5664864864864865 : 0.8385877711529933
n_estimators = 12 : 0.5654008438818565 : 0.837490999318094
n_estimators = 13 : 0.5659163987138264 : 0.8362591179093304
n_estimators = 14 : 0.55863539445629 : 0.8353324251850605
n_estimators = 15 : 0.5529661016949152 : 0.8352767918311161


Лучший n_estimators = 7 (F1=0.57).

In [39]:
for i in range(1,16):
    model = RandomForestClassifier(random_state=12345,max_depth=i, 
                                   n_estimators=7)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_depth = {i} : {f1} : {auc_roc}')

max_depth = 1 : 0.4224643755238894 : 0.7499733754663269
max_depth = 2 : 0.5076765609007166 : 0.8048914434312111
max_depth = 3 : 0.50355871886121 : 0.8050925182676091
max_depth = 4 : 0.5561497326203209 : 0.8146884770607786
max_depth = 5 : 0.5570680628272251 : 0.8292000400560148
max_depth = 6 : 0.5720524017467249 : 0.8306989615637105
max_depth = 7 : 0.5579399141630901 : 0.8365587432584297
max_depth = 8 : 0.5551982851018221 : 0.838484452067097
max_depth = 9 : 0.5541125541125542 : 0.8357377539066536
max_depth = 10 : 0.5578831312017641 : 0.8285236974244936
max_depth = 11 : 0.5407925407925408 : 0.8194022116642481
max_depth = 12 : 0.5586854460093897 : 0.8185708904038822
max_depth = 13 : 0.5502392344497609 : 0.8082159337104744
max_depth = 14 : 0.5612745098039216 : 0.8255075748785206
max_depth = 15 : 0.5538461538461539 : 0.8161778614220838


На данном этапе лучший лес (f1=0.57) имеет гиперпараметры max_depth=6 и n_estimators=7. У этих гиперпараметров auc_roc=0.83. Пороговое значения f1 (0.59) не достигнуто.

Проверим логистическую регрессию. 

In [40]:
for i in range(100,1700,100):
    model = LogisticRegression(random_state=12345, solver='liblinear', 
                               max_iter=i)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
    print(f'max_iter = {i} : {f1} : {auc_roc}')

max_iter = 100 : 0.4771126760563381 : 0.7725692595518495
max_iter = 200 : 0.4771126760563381 : 0.7725692595518495
max_iter = 300 : 0.4771126760563381 : 0.7725692595518495
max_iter = 400 : 0.4771126760563381 : 0.7725692595518495
max_iter = 500 : 0.4771126760563381 : 0.7725692595518495
max_iter = 600 : 0.4771126760563381 : 0.7725692595518495
max_iter = 700 : 0.4771126760563381 : 0.7725692595518495
max_iter = 800 : 0.4771126760563381 : 0.7725692595518495
max_iter = 900 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1000 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1100 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1200 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1300 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1400 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1500 : 0.4771126760563381 : 0.7725692595518495
max_iter = 1600 : 0.4771126760563381 : 0.7725692595518495


Пороговое значения f1 ( 0.59) не достигнуто.

## Изменение порога случайного леса

Попробуем ещё одну технику. Изменим порог вероятности классов модели и посмотрим на значения F1 меры. Работу также будем продолжать со сбалансированной выборкой.  

Так как на прошлых этапах ни модель решающего дерева, ни модель логистической регрессии не показали заметного прогресса, мы решили сосредоточить наши усилия на модели случайного леса. 

Напишем цикл который создаст 4 списка, куда поместим значения гиперпараметров, f1 меры и значение порога вероятности класса. 

In [41]:
max_depth_list=[]
n_estimators_list=[]
threshold_list=[]
f1_list=[]
for i in range(7,17):
    for e in range(7,17):
        for threshold in np.arange(0.4, 0.7, 0.05):
            model = RandomForestClassifier(random_state=12345,max_depth=i, 
                                   n_estimators=e)
            model.fit(features_upsampled, target_upsampled)
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            predicted_valid = probabilities_one_valid > threshold
            f1 = f1_score(target_valid, predicted_valid)
            max_depth_list.append(i)
            n_estimators_list.append(e)
            threshold_list.append(threshold)
            f1_list.append(f1)
print('выполнено')

выполнено


Теперь соберём из этих списков новый датафрейм. И выведем значения, где f1 мера больше 0.6

In [42]:
dict = {'f1': f1_list, 'max_depth': max_depth_list, 
        'n_estimators': n_estimators_list, 'threshold': threshold_list} 
df = pd.DataFrame(dict)
print(df[df['f1']>=0.6])

          f1  max_depth  n_estimators  threshold
10  0.609819          7             8       0.60
16  0.610825          7             9       0.60
28  0.606771          7            11       0.60
34  0.610966          7            12       0.60
39  0.607565          7            13       0.55
45  0.610849          7            14       0.55
51  0.612676          7            15       0.55
52  0.600529          7            15       0.60
57  0.602100          7            16       0.55
58  0.602378          7            16       0.60


Мы считаем что нужно сосредоточиться на более низких значениях max_depth и n_estimators, чтобы не было переобучения модели. Но и значение f1 меры для нас важно. Поэтому мы считаем правильным использовать значения геперпараметров max_depth=7 и n_estimators=9, и значение порога вероятности класса = 0.6

## Тестирование модели

Проверим нашу модель на тестовой выборке. В качестве обучающей выборки будем также использовать сбалансированную выборку.

In [43]:
model = RandomForestClassifier(random_state=12345,max_depth=7, 
                                   n_estimators=9)
model.fit(features_upsampled, target_upsampled)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.6
auc_roc= roc_auc_score(target_test, probabilities_one_test)
f1 = f1_score(target_test, predicted_test)
print('f1=', f1)
print('auc_roc=', auc_roc)

f1= 0.6375757575757576
auc_roc= 0.8629224724604754


Наша модель достигла показателя выше минимума f1=0.59. Показатель auc_roc также оказался высоким.

## Вывод

В нашем исследовании нам были предоставлены данные о клиентах банка, где в том числе был указан факт отказа части клиентов от услуг этого банка. Наша цель была создать модель, которая сможет в будущем на основе новых данных о других клиентах предсказывать откажется ли данный клиент от услуг банка.

Нам был установлен минимальный порог F1 меры для нашей модели равный 0,59. 

Для создания модели мы провели предобработку данных. Отказались от части данных, которые посчитали излишними для модели, хранящихся в столбцах: 

    RowNumber — индекс строки в данных
    CustomerId — уникальный идентификатор клиента
    Surname — фамилия.
    
Данные в столбце Tenure (сколько лет человек является клиентом банка) содержали пропуски. Мы решили не удалять эти данные, а заполнить их медианным значением. Таким образом в будущем наша модель также сможет обрабатывать подобные данные, их не придётся исключать.

Также для финальной модели данные были сбалансированы техникой upsampled. Категориальные данные из столбцов Geography и Gender были обработаны техникой OHE(используя параметр drop_first=True). Количественные признаки CreditScore, Age, Tenure, Balance и EstimatedSalary были  масштабируемы. 

Наше исследование пришло к тому, что наиболее эффективной, с точки зрения  F1 меры получилась модель Случайный лес с значениями геперпараметров max_depth=7 и n_estimators=9, и значением  порога вероятности класса = 0.6. 

В качестве обучающих данных для этой модели мы использовали сбалансированные выборки хранящиеся в features_upsampled и target_upsampled.

Проверка финальной модели на тестовой выборке показала следующие результаты:

	F1 = 0.636
    AUC ROC = 0.862
    
Наша модель достигла показателя выше минимума F1=0.59. Показатель AUC ROC также оказался высоким.